### Import Packages


In [87]:
import torch
import numpy as np
import time
from scipy.ndimage import distance_transform_edt as distance
from scipy.ndimage import _nd_image

In [88]:
device = torch.device("cuda:0")
print(device)

cuda:0


## Benchmarks

In [89]:
# 2D
dt_2D_mean = []
dt_2D_std = []
	
dt_2D_GPU_mean = []
dt_2D_GPU_std = []

dt_2D_times = []
dt_2D_GPU_times = []

In [90]:
# 2D on 3D image
dt_2v3D_mean = []
dt_2v3D_std = []
	
dt_2v3D_GPU_mean = []
dt_2v3D_GPU_std = []

dt_2v3D_times = []
dt_2v3D_GPU_times = []

In [91]:
#3D
dt_3D_mean = []
dt_3D_std = []
	
dt_3D_GPU_mean = []
dt_3D_GPU_std = []

dt_3D_times = []
dt_3D_GPU_times = []

In [92]:
#2D times
for n in range(2,210,10):
  # Euclidean Distance CPU
  temp_dt_time = []
  for i in range(100): #Samples
    a = []
    for j in range(10): #Evaluations
      x1 = torch.randint(0,2,(n,n))
      times1 = time.time()
      tfm1 = torch.from_numpy(distance(x1))
      a.append(time.time()-times1)
    temp_dt_time.append(torch.mean(torch.FloatTensor(a)))
  dt_2D_times.append(temp_dt_time)


# for n in range(2,210,10):
#   # Euclidean Distance GPU
#   temp_dt_GPU_time = []
#   for i in range(100): #Samples
#     b = []
#     for j in range(10): #Evaluations
#       x2 = torch.randint(0,2,(n,n)).to(device)
#       times2 = time.time()
#       tfm2 = torch.from_numpy(distance(x2.cpu().numpy())).to(device)  
#       b.append(time.time()-times2)
#     temp_dt_GPU_time.append(torch.mean(torch.FloatTensor(b)))
#   dt_2D_GPU_times.append(temp_dt_GPU_time)


In [93]:
# 2D times on 3D image
for n in range(2,210,10):
  # Euclidean Distance CPU
  temp_dt_time = []
  for i in range(100): #Samples
    a = []
    for j in range(10): #Evaluations
      x1 = torch.randint(0,2,(n,n,n))
      times1 = time.time()
      for k in range(n):
        slice = x1[:,:,k]
        tfm1 = torch.from_numpy(distance(slice))
      a.append(time.time()-times1)
    temp_dt_time.append(torch.mean(torch.FloatTensor(a)))
  dt_2v3D_times.append(temp_dt_time)


# for n in range(2,210,10):
#   # Euclidean Distance GPU
#   temp_dt_GPU_time = []
#   for i in range(100): #Samples
#     b = []
#     for j in range(10): #Evaluations
#       x2 = torch.randint(0,2,(n,n,n)).to(device)
#       times2 = time.time()
#       for k in range(n):
#         slice = x2[:,:,k]
#         tfm2 = torch.from_numpy(distance(slice.cpu().numpy())).to(device)  
#       b.append(time.time()-times2)
#     temp_dt_GPU_time.append(torch.mean(torch.FloatTensor(b)))
#   dt_2v3D_GPU_times.append(temp_dt_GPU_time)


In [94]:
#3D times
for n in range(2,210,10):
  # Euclidean Distance CPU
  temp_dt_time = []
  for i in range(100): #Samples
    a = []
    for j in range(10): #Evaluations
      x1 = torch.randint(0,2,(n,n,n))
      times1 = time.time()
      tfm1 = torch.from_numpy(distance(x1))
      a.append(time.time()-times1)
    temp_dt_time.append(torch.mean(torch.FloatTensor(a)))
  dt_3D_times.append(temp_dt_time)


# for n in range(2,210,100):
#   # Euclidean Distance GPU
#   temp_dt_GPU_time = []
#   for i in range(100): #Samples
#     b = []
#     for j in range(10): #Evaluations
#       x2 = torch.randint(0,2,(n,n,n)).to(device)
#       times2 = time.time()
#       tfm2 = torch.from_numpy(distance(x2.cpu().numpy())).to(device)  
#       b.append(time.time()-times2)
#     temp_dt_GPU_time.append(torch.mean(torch.FloatTensor(b)))
#   dt_3D_GPU_times.append(temp_dt_GPU_time)


In [95]:
#2D means
for i in dt_2D_times:
  dt_2D_mean.append(torch.mean(torch.FloatTensor(i)).numpy().tolist())
  dt_2D_std.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

In [96]:
#2D means on 3D image
for i in dt_2v3D_times:
  dt_2v3D_mean.append(torch.mean(torch.FloatTensor(i)).numpy().tolist())
  dt_2v3D_std.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

In [97]:
#3D means
for i in dt_3D_times:
  dt_3D_mean.append(torch.mean(torch.FloatTensor(i)).numpy().tolist())
  dt_3D_std.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

In [98]:
#2D means GPU
for i in dt_2D_GPU_times:
  dt_2D_GPU_mean.append(torch.mean(torch.FloatTensor(i)).numpy().tolist())
  dt_2D_GPU_std.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

In [99]:
#2D means 3D image GPU
for i in dt_2v3D_GPU_times:
  dt_2v3D_GPU_mean.append(torch.mean(torch.FloatTensor(i)).numpy().tolist())
  dt_2v3D_GPU_std.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

In [100]:
#3D means GPU
for i in dt_3D_GPU_times:
  dt_3D_GPU_mean.append(torch.mean(torch.FloatTensor(i)).numpy().tolist())
  dt_3D_GPU_std.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

## Save

In [101]:
import pandas as pd

In [102]:
dt_2D_mean = np.array(dt_2D_mean)
dt_2D_GPU_mean = np.array(dt_2D_GPU_mean)
dt_2D_std = np.array(dt_2D_std)
dt_2D_GPU_std = np.array(dt_2D_GPU_std)
dt_2v3D_mean = np.array(dt_2v3D_mean)
dt_2v3D_GPU_mean = np.array(dt_2v3D_GPU_mean)
dt_2v3D_std = np.array(dt_2v3D_std)
dt_2v3D_GPU_std = np.array(dt_2v3D_GPU_std)
dt_3D_mean = np.array(dt_3D_mean)
dt_3D_GPU_mean = np.array(dt_3D_GPU_mean)
dt_3D_std = np.array(dt_3D_std)
dt_3D_GPU_std = np.array(dt_3D_GPU_std)

In [103]:
data2D = {'dt_mean_cpu_2D': dt_2D_mean, 'dt_std_cpu_2D': dt_2D_std}


# data2D = {'dt_mean_cpu_2D': dt_2D_mean, 'dt_std_cpu_2D': dt_2D_std, 'dt_mean_2D': dt_2D_GPU_mean, 'dt_std_gpu_2D': dt_2D_GPU_std
#  }

In [104]:
data2v3D = {'dt_mean_cpu_2v3D': dt_2v3D_mean, 'dt_std_cpu_2v3D': dt_2v3D_std}

# data2v3D = {'dt_mean_cpu_2v3D': dt_2v3D_mean, 'dt_std_cpu_2v3D': dt_2v3D_std, 'dt_mean_2v3D': dt_2v3D_GPU_mean, 'dt_std_gpu_2v3D': dt_2v3D_GPU_std}

In [105]:
data3D = {'dt_mean_cpu_3D': dt_3D_mean, 'dt_std_cpu_3D': dt_3D_std}

# data3D = {'dt_mean_cpu_3D': dt_3D_mean, 'dt_std_cpu_3D': dt_3D_std, 'dt_mean_3D': dt_3D_GPU_mean, 'dt_std_gpu_3D': dt_3D_GPU_std}

In [106]:
dataframe2D = pd.DataFrame(data2D)
dataframe2v3D = pd.DataFrame(data2v3D)
dataframe3D = pd.DataFrame(data3D)

In [107]:
dataframe2D.to_csv("G:\molloi-lab\project-distance-transforms\performance\purePython_DT_May03_2D.csv")
dataframe2v3D.to_csv("G:\molloi-lab\project-distance-transforms\performance\purePython_DT_May03_2v3D.csv")
dataframe3D.to_csv("G:\molloi-lab\project-distance-transforms\performance\purePython_DT_May03_3D.csv")

In [108]:
dataframe2v3D

,dt_mean_cpu_2v3D,dt_std_cpu_2v3D
0,0.000070,0.000052
1,0.000500,0.000051
2,0.001321,0.000086
3,0.002877,0.000087
4,0.005544,0.000261
5,0.009868,0.000191
6,0.015548,0.000143
7,0.023501,0.000154
8,0.033934,0.000237
9,0.046775,0.000192


In [109]:
dataframe3D

,dt_mean_cpu_3D,dt_std_cpu_3D
0,0.000027,0.000044
1,0.000189,0.000049
2,0.001010,0.000036
3,0.003246,0.000137
4,0.007342,0.000196
5,0.015117,0.000205
6,0.025730,0.000926
7,0.041166,0.000457
8,0.060666,0.000407
9,0.086277,0.000456


In [110]:
x = [i for i in range(1, 1000, 100)]

In [111]:
import matplotlib.pyplot as plt

In [112]:
# plt.figure(figsize=(13, 13))
# plt.plot(x, dataframe['dt_mean_cpu_3D'], label = 'dt_mean_cpu_3D')
# plt.plot(x, dataframe['dt_mean_gpu_3D'], label = 'dt_mean_gpu_3D')
# plt.xlabel('Array_Size')
# plt.ylabel('Time (seconds)')
# plt.legend()
# plt.show()